In [ ]:
import sys
!{sys.executable} -m pip install duckdb==0.7.1 notebook pandas ipython-sql SQLAlchemy duckdb-engine altair

In [ ]:
import duckdb
import pandas as pd
import sqlalchemy

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

In [9]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [10]:
%sql duckdb:///:memory:

In [11]:
%%sql

INSTALL httpfs;
LOAD httpfs;
SET s3_endpoint='my-cluster.us-east-2.lakefscloud.io';
SET s3_access_key_id='access_key';
SET s3_secret_access_key='secret';
SET s3_url_style='path';
SET s3_region='us-east-1';

""


In [12]:
%%sql

CREATE TABLE dim_asset AS 
	SELECT * FROM read_parquet('s3://drones03/alice-test/drone-registrations/star/dim_asset.parquet');
CREATE TABLE dim_location AS 
	SELECT * FROM read_parquet('s3://drones03/alice-test/drone-registrations/star/dim_location.parquet');
CREATE TABLE fact_registrations AS 
	SELECT * FROM read_parquet('s3://drones03/alice-test/drone-registrations/star/fact_registrations.parquet');


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,875000


## What's the most popular drone model to be registered? 

In [13]:
%%sql

SELECT model,count(*) 
FROM dim_asset a 
     left join fact_registrations r on a.asset_id=r.asset_id 
group by model 
order by 2 desc limit 25;

,model,count_star()
0,Mavic 2 Pro,14505
1,Mavic Air 2,11740
2,Mavic Air,7353
3,Mavic Pro,7136
4,Air 2S,6953
5,Mini 2,6778
6,X1,6725
7,Spark,5108
8,Mavic 2 Zoom,4354
9,Phantom 4 Pro,4129


Check out that data quality…or lack thereof. We've got `Mavic` and `MAVIC`, we've got `Mavic Air 2S` alongside `AIR 2S`. Let's fix that so that we get accurate numbers. 

Let's take the example of the [Air 2S](https://www.dji.com/uk/air-2s). I'm dropping the `DJI` prefix from all of them since this is the manufacturer. 

In [14]:
%%sql

SELECT model,count(*) FROM dim_asset a left join fact_registrations r on a.asset_id=r.asset_id 
	 where lower(model) like '%air%' 
	   and lower(model) like '%2s%' 
	group by model order by 2 desc 
	limit 30;


,model,count_star()
0,Air 2S,6953
1,Mavic Air 2S,3608
2,AIR 2S,3568
3,Air 2s,3439
4,DJI Air 2S,1217
5,Mavic Air 2s,983
6,DJI AIR 2S,842
7,MAVIC AIR 2S,568
8,Air2S,511
9,Air2s,501


This is just the first 30 permutations. What if we group them based on the lowercase representation of the field: 

In [15]:
%%sql

SELECT lower(model) as model,count(*) FROM dim_asset a left join fact_registrations r on a.asset_id=r.asset_id 
	 where lower(model) like '%air%' 
	   and lower(model) like '%2s%' 
	group by lower(model) order by 2 desc 
	limit 30;


,model,count_star()
0,air 2s,14388
1,mavic air 2s,5538
2,dji air 2s,2899
3,air2s,1421
4,mavic air2s,187
5,dji mavic air 2s,177
6,dji air2s,177
7,magic air 2s,45
8,mavik air 2s,20
9,mavis air 2s,17


This catches most of them. Let's add a field to the `dim_asset` table for cleaning this data up, which by default will be the same value as `model`. 

In [ ]:
%%sql

ALTER TABLE dim_asset ADD COLUMN model_cleaned VARCHAR;
UPDATE dim_asset SET model_cleaned = model;

Now we can clean up the Air 2S references: 

In [17]:
%%sql

UPDATE dim_asset
   SET model_cleaned = 'Air 2S'
 WHERE lower(model) LIKE '%air 2s%'
    OR lower(model) LIKE '%air2s%';

,Count
0,438


Let's try our original "most popular" query again

In [18]:
%%sql

SELECT model_cleaned,count(*) 
  FROM dim_asset a 
  left join fact_registrations r on a.asset_id=r.asset_id 
  group by model_cleaned 
  order by 2 desc 
  limit 5;

,model_cleaned,count_star()
0,Air 2S,25165
1,Mavic 2 Pro,19091
2,Mavic Air 2,19085
3,Mavic Air,10525
4,Mavic Pro,9422


Compare that to the table above, where `Air 2S` came fifth in the list with ~6k registrations. Data quality matters! Let's clean up a few more, and see what that does

In [19]:
%%sql

UPDATE dim_asset
   SET model_cleaned = 'Mavic 2 Pro'
 WHERE lower(model) LIKE '%mavic 2 pro%';

UPDATE dim_asset
   SET model_cleaned = 'Mavic Air 2'
 WHERE lower(model) LIKE '%air 2';

UPDATE dim_asset
   SET model_cleaned = 'Mavic Air'
 WHERE lower(model) LIKE '%air' 
   and lower(model) NOT LIKE '%corsair%';
   
UPDATE dim_asset
   SET model_cleaned = 'Mavic Pro'
  WHERE lower(model) LIKE '%pro' 
    AND lower(model) LIKE '%mavic%'
    AND lower(model) NOT LIKE '%2%'
    AND lower(model) NOT LIKE '%3%'
    AND lower(model) NOT LIKE '%mini%';


,Count
0,195


In [20]:
%%sql

SELECT model_cleaned,count(*)  as registrations
FROM dim_asset a 
     left join fact_registrations r on a.asset_id=r.asset_id 
group by model_cleaned 
order by 2 desc limit 10;

,model_cleaned,registrations
0,Air 2S,25165
1,Mavic 2 Pro,19091
2,Mavic Air 2,19085
3,Mavic Air,10525
4,Mavic Pro,9422
5,Mini 2,6778
6,X1,6725
7,Spark,5108
8,Mavic 2 Zoom,4354
9,Phantom 4 Pro,4129


In [21]:
%%sql

top_drones_df << SELECT model_cleaned as model,count(*) as registrations
FROM dim_asset a 
     left join fact_registrations r on a.asset_id=r.asset_id 
group by model_cleaned 
order by 2 desc;

Returning data to local variable top_drones_df


In [ ]:
import sys
!{sys.executable} -m pip install altair

In [23]:
import altair as alt

chart = alt.Chart(top_drones_df.head(10)).mark_bar().encode(
   x=alt.X('model', axis=alt.Axis(ticks=False, labels=True, title='')),
   y=alt.Y('registrations', axis=alt.Axis(grid=False)),
   color='model'
).configure_view(
    stroke=None,
)

chart.display()

alt.Chart(...)

In [ ]:
%%sql 

CREATE TABLE TOP_DRONES AS 
SELECT model_cleaned AS model,count(*)  as registrations
FROM dim_asset a 
     left join fact_registrations r on a.asset_id=r.asset_id 
group by model_cleaned 
order by 2 desc 


,Count
0,33680


In [ ]:
%%sql
COPY TOP_DRONES TO 's3://drones03/alice-test/drone-registrations/star/top_drones.parquet';



FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,33680


## Analyse registrations per state

In [ ]:
import sys
!{sys.executable} -m pip install plotly-express

In [25]:
%%sql

location_reg_df << 
SELECT STATE_PROVINCE as state_code,count(*) as registrations
FROM dim_location l 
     left join fact_registrations r on l.location_id=r.physical_location_id
group by STATE_PROVINCE
order by 2 desc;

Returning data to local variable location_reg_df


In [32]:
# ref https://towardsdatascience.com/simplest-way-of-creating-a-choropleth-map-by-u-s-states-in-python-f359ada7735e
import plotly.express as px
fig = px.choropleth(location_reg_df,
                    locations='state_code', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='registrations',
                    color_continuous_scale="speed", 
                                        )

fig.update_layout(
      title_text = 'Drone registrations 2016-2021',
      title_font_family="Arial",
      title_font_size = 22,
      title_font_color="black", 
      title_x=0.45, 
         )

fig.show()

## Write the modified data back to lakeFS

In [28]:
%%sql

COPY dim_asset TO 's3://drones03/alice-test/drone-registrations/star/dim_asset.parquet' (FORMAT PARQUET, ALLOW_OVERWRITE TRUE);


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,43015


What version of DuckDB is being used?

In [29]:
%%sql

select version()

,version()
0,0.7.1
